# Images to .mpeg Python Script 
### This code example processes a sequence of images using multiple steps to create an MPEG movie as a result. 
#### The comments provide additional context for each command.
First import necessary modules to run the script. <font color='purple'>Image</font>  allows you to open the image sequence required to create the mpeg. <font color='purple'>Math</font> allows you to do various math functions with python. <font color='purple'>OS</font> enables operating system dependent functionality. <font color='purple'>ImageIO</font> enables you to convert the final set of 2D images into an mpeg

In [1]:
from PIL import Image
import math
import os
import imageio

This section reads in a sequence of images,extracts a square region, and resaves them as new images.

In [ ]:
# Identify the location of the images in the image sequence
dir = 'C:/Users/anjal/Downloads/create_mpeg_from_image_sequence/create_mpeg_from_image_sequence/25x25simple/png_rect_subtract_min_norm_blur/'
# Create a variable that can be traversed for each image in dir
filenames = next(os.walk(dir))[2]
for filename in filenames: # Traverse through all the filenames (or images) in dir
    print("Working on file:" + filename) # Print a progress statement

    im = Image.open(dir + filename) # Can be many different formats.
    pix = im.load() #Load the image
    print(im.size)  # Get the width and height of the image for iterating over and display it to the user
    hw = im.size # Create a variable that will store the size of the image
    width = hw[0] # Obtain the width from the size list
    height = hw[1] # Obtain the height from the size list
    
    im2 = Image.open('C:/Users/anjal/Downloads/create_mpeg_from_image_sequence/create_mpeg_from_image_sequence/25x25simple/g2_square_535x535.png') # Can be many different formats.
    pix2 = im2.load() # Load a blank image of the desired dimensions (In this case 535x535)
    print(im2.size)  # Obtain the width and height of the image for iterating over
    
    # Copy the RGBA value of each pixel within the desired range of the image onto the blank image of desired dimensions
    for i in range(257,792):
        for j in range(208,743):
            x = i - 257
            y = j - 208
            pix2[x,y] = pix[i,j] 
                   
    # Save the modified pixels as .png
    im2.save('C:/Users/anjal/Downloads/create_mpeg_from_image_sequence/create_mpeg_from_image_sequence/25x25simple/squares_subtract_min_norm/' + filename) 


Now the cropped image must be projected onto the final template to ultimately form the mpeg file

In [ ]:
# Access the directory in which the cropped images are found
dir = 'C:/Users/anjal/Downloads/create_mpeg_from_image_sequence/create_mpeg_from_image_sequence/25x25simple/squares_subtract_min_norm/'

inc = 0
filenames = next(os.walk(dir))[2] # Create a variable that can be traversed for each image in dir
for filename in filenames: # Traverse through the filenames (or images) in the final image directory
    print("Creating Final 2D Map for:" + filename) # Print a progress statement
    im = Image.open(dir + filename) # Can be many different formats.
    pix = im.load() # Load the image
    print(im.size)  # Get the width and hight of the image for iterating over
    hw = im.size # Create a variable that will store the size of the image
    width = hw[0] # Obtain the width from the size list
    height = hw[1] # Obtain the height from the size list
    half_height = round(0.5*height)
    half_width = round(0.5*width)
    
    im2 = Image.open('C:/Users/anjal/Downloads/create_mpeg_from_image_sequence/create_mpeg_from_image_sequence/25x25simple/g2_delta_lr_mean_template.png') # Can be many different formats.
    pix2 = im2.load() # Load the template image that will appear in the final mpeg
    
    for i in range(-half_width,half_width-1): # Loop through x and y coordinates
        for j in range(-half_height,half_height-1):
            radius_var = math.sqrt(i*i + j*j) # Calculate distance from center of image to pixel
            x = i + half_width 
            y = j + half_height
            x2 = x + 266 # Loops through all x and y coordinates within final circular region
            y2 = y + 54
            
            if radius_var < half_height: # Check if the pixel is within the radius of the circular region of radius half_height
                final_i = x # Extract the RGBA values for each pixel within the circular region
                final_j = y
    
                if final_i > width-1:
                    final_i = width-1
                if final_j > width-1:
                    final_j = width-1
                
                pix2[x2,y2] = pix[final_i,final_j] # Copy the RGBA value of the original image onto the template
                
    i_start = 249 + round(inc * 660/15400) # Identify x-value starting place of vertical line on template
    
    for i in range(i_start, i_start+2): # Loop through width of line
        for j in range(760,939): # Loop through height of line for the first graph
            pix2[i,j] = 0,0,0,255 # Make pixels in width and height range black 
        for j in range(990,1164): # Loop through height of line for the second graph (below first graph)
            pix2[i,j] = 0,0,0,255
    inc += 1
    # Save the modified template as .png
    im2.save('C:/Users/anjal/Downloads/create_mpeg_from_image_sequence/create_mpeg_from_image_sequence/25x25simple/circles_subtract_min_norm/' + filename) 

Finally the set of final images must be converted to an mpeg file using imageio

In [ ]:
imageio.plugins.ffmpeg.download() # Download the necessary plugin from the imageio module
images = [] # Create empty images list

# Access the directory in which the final images are found
dir = 'C:/Users/anjal/Downloads/create_mpeg_from_image_sequence/create_mpeg_from_image_sequence/25x25simple/circles_subtract_min_norm/'

filenames = next(os.walk(dir))[2] # Create a variable that can be traversed for each filename (or image) in dir
for filename in filenames: # Traverse through the final images
    file_input = dir + filename #Identify the location of the final image
    images.append(imageio.imread(file_input)) # Add each final image file to the images list
    
imageio.mimsave('C:/Users/anjal/Downloads/create_mpeg_from_image_sequence/create_mpeg_from_image_sequence/25x25simple/movies/25x25_subtract_min_norm.mp4', images)
# Convert the images list into an mpeg file and save it 